# EOF analysis of TG data

## Learning Goals

* Fundamentals of Principal Component Analysis in climate science (EOFs)
* Matrix operations using numpy
* Defining a function in Python
* Create a Dataset from scratch in Xarray 

### Import Packages

In [2]:
%matplotlib widget
import os                         
import xarray as xr 
import pandas as pd
import matplotlib                 
import matplotlib.pyplot as plt   
import cartopy                  
import cartopy.crs as ccrs      
import plotly.express as px
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')
import numpy as np
import panel as pn
import panel.widgets as pnw
import ipywidgets as ipw
from sklearn import datasets, linear_model
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score
import glob2
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import cartopy.crs as ccrs

<head>
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css" rel="stylesheet">
</head>

<head>
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css" rel="stylesheet">
</head>

### Upload the Data

Uploading the dataset we created and saved in the previous chapter. 

In [3]:
path = '/Users/ynorden/Research/code/coastal-sl/data/'
tg_ds=xr.open_dataset(path+'ds_tg.nc')

In [4]:
tg_ds.hvplot.scatter(x='lon', y='lat', c='id',geo=True, coastline=True)

:DynamicMap   [time]
   :Overlay
      .Scatter.I   :Scatter   [lon]   (lat,id)
      .Coastline.I :Feature   [Longitude,Latitude]

## EOF analysis 

EOFs (Empirical Orthogonal Functions) are a widely used statistical technique in climate science and other  fields of earth science data analysis. \
It is a type of dimensionality reduction technique that retains modes of variability present in a given dataset. \
The key characteristic of EOFs is that they are orthogonal to each other, meaning they are statistically independent. This property allows each EOF to capture a unique and independent pattern of variability in the dataset.\


```{admonition} Important
:class: tip-yellow
:icon: fas fa-lightbulb
   
EOF analysis is a purely mathematical technique and does not imply anything about the physical mechanisms of the patterns that arise. It is up to a researcher to interpret the results. 

<style>
    .bright-yellow-cell {
        background-color: yellow;
        color: black;
        font-weight: bold;
        padding: 8px;
        border: 1px solid black;
        border-radius: 4px;
    }
</style>

## EOF function

In [5]:
print(tg_ds.sla.shape)

(31, 9784)


In [6]:
# because I keep tide gauges time-series and metadata such as 
def eof_tg(tg_ds):
    '''
    because I keep tide gauges time-series and metadata such as station id in separate df's 
    that's why I need these two df's as input 
    Example. my tg_data.shape : (9784,33)
                tg_meta.shape : (33,5)
    '''
    data = tg_ds.sla.T.data
    nt = len(tg_ds.time)
    covmat = (data.T @ data) / nt  #covariance matrix
    eigvals, eigvecs = np.linalg.eig(covmat)
    pc = data @ eigvecs
    
    c = len(tg_ds.id)+1
    eof_names = ['EOF%s' % k for k in range(1, c)]
    pc_names = ['PC%s' % l for l in range(1, c)]
    eof_df = pd.DataFrame(eigvecs, index=tg_ds.id, columns=eof_names)
    pc_df = pd.DataFrame(pc, index=tg_ds.time, columns=pc_names)

    data_from_eof_df = pc_df.values @ eof_df.T.values
    data_from_eof_df = pd.DataFrame(data_from_eof_df, index=tg_ds.time, columns=tg_ds.id)

    return eof_df, pc_df, data_from_eof_df

In [7]:
eof_df, pc_df, data_from_eof_df = eof_tg(tg_ds)

In [8]:
pc_ds = xr.Dataset({'pc_n':(('time','pc'),pc_df.values),
                    },
                        {'time':tg_ds.time, 'pc': np.arange(1,tg_ds.id.size+1)},
)
#maybe I better create an xarray dataarray 

In [9]:
# saving a dataset to use in the next chapter  
#pc_ds.to_netcdf(path+'pc_ds.nc')

This whole section should be time for me to work of pythia cookbook material to refer people to it when it's done. I kind of should understand what eigvecs and eigvals are better, include variance of each mode and then variance explained at each location section. 

### Plots

#### Spatial Patterns

In [10]:
eof_df_g = eof_df.copy()
eof_df_g['lat']=tg_ds.lat.data
eof_df_g['lon']=tg_ds.lon.data

In [11]:
eof_df_g 

,EOF1,EOF2,EOF3,EOF4,EOF5,EOF6,EOF7,EOF8,EOF9,EOF10,...,EOF24,EOF25,EOF26,EOF27,EOF28,EOF29,EOF30,EOF31,lat,lon
8410140,-0.111759,-0.064118,0.151425,-0.193984,0.303376,-0.001483,0.272311,0.058981,0.040844,0.010152,...,-0.104914,-0.289350,0.059312,-0.124085,0.054359,0.076516,0.020430,0.007549,44.904598,-66.982903
8418150,-0.141232,-0.112236,0.181463,-0.148933,0.249461,-0.115628,0.166304,0.069320,0.048500,-0.025495,...,0.171640,0.136686,-0.274324,0.397766,-0.133845,0.002612,0.210897,-0.176941,43.658060,-70.244170
8443970,-0.188163,-0.109269,0.169621,-0.172418,0.222142,-0.035318,0.101072,0.110958,0.047713,0.010957,...,-0.176911,-0.044033,0.282733,-0.428173,0.197988,-0.062411,-0.150792,0.350241,42.354801,-71.053400
8447930,-0.183028,-0.088591,0.139571,-0.124425,0.166466,-0.060155,0.029810,0.019439,0.027926,0.061553,...,0.258986,0.354732,-0.143197,-0.056737,0.050288,0.409783,0.063988,0.401042,41.523613,-70.671112
8449130,-0.181594,-0.092365,0.100638,-0.203448,0.184764,-0.018550,0.177009,0.093149,0.039548,0.273261,...,0.066640,-0.052739,-0.060813,0.147265,-0.074266,0.016855,0.040109,-0.320175,41.285278,-70.096390
8452660,-0.186126,-0.098641,0.141271,-0.085431,0.125278,-0.083479,-0.023434,0.021854,0.015917,-0.070373,...,0.063412,0.171363,0.589761,0.156862,-0.283870,-0.466514,-0.135206,-0.110736,41.504333,-71.326139
8461490,-0.212953,-0.112007,0.145516,-0.061612,0.076055,-0.059158,-0.102719,0.062262,0.005895,-0.201495,...,-0.285042,-0.084026,-0.602158,-0.014221,0.199944,-0.378919,-0.278380,-0.019598,41.371700,-72.095525
8467150,-0.228104,-0.135224,0.139187,0.026386,-0.036037,-0.164821,-0.220339,0.068080,-0.038363,-0.273836,...,0.056921,-0.393664,0.158711,-0.049294,-0.038828,0.433199,0.254297,-0.155450,41.173302,-73.181702
8518750,-0.243364,-0.116148,0.085427,0.079643,-0.109172,-0.104912,-0.341069,-0.043449,-0.050610,-0.144254,...,-0.008219,0.058985,0.021188,0.004323,-0.006924,0.004990,-0.030387,0.018702,40.700556,-74.014167
8531680,-0.277924,-0.111745,0.072979,0.053675,-0.091611,-0.046845,-0.286315,-0.019356,-0.014532,0.149509,...,0.005029,0.077380,-0.003023,0.007223,0.018565,-0.022463,0.016033,-0.018677,40.466944,-74.009444


In [12]:
eig_eof_1 = eof_df_g.hvplot.points('lon','lat',c='EOF1',geo=True,projection=ccrs.PlateCarree(),
                                    coastline=True,cmap='Spectral',line_width=5, title = 'EOF1: spatial pattern of variability',colorbar=True)
eig_eof_2 = eof_df_g.hvplot.points('lon','lat',c='EOF2',geo=True,projection=ccrs.PlateCarree(),
                                    coastline=True,cmap='Spectral',line_width=5, title = 'EOF2: spatial pattern of variability',colorbar=True)
eig_eof_3 = eof_df_g.hvplot.points('lon','lat',c='EOF3',geo=True,projection=ccrs.PlateCarree(),
                                    coastline=True,cmap='Spectral',line_width=5, title = 'EOF3: spatial pattern of variability',colorbar=True)
eig_eof_4 = eof_df_g.hvplot.points('lon','lat',c='EOF4',geo=True,projection=ccrs.PlateCarree(),
                                    coastline=True,cmap='Spectral',line_width=5, title = 'EOF4: spatial pattern of variability',colorbar=True)

#### Principal Components

In [13]:
pc_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31
1993-01-01,0.129575,-0.057145,0.021785,-0.037181,-0.046846,0.308842,0.109787,-0.029381,-0.004198,0.010261,...,0.000545,-0.006427,0.001542,0.039090,0.027648,-0.004983,-0.014321,-0.026200,0.028805,-0.000147
1993-01-02,0.457598,0.322300,-0.374299,0.041988,-0.021173,0.302031,-0.032742,0.018476,0.060944,-0.028407,...,0.000033,-0.024126,0.027133,0.052957,0.025456,-0.002528,-0.049843,-0.022028,-0.020625,-0.006009
1993-01-03,0.140078,0.371345,-0.173575,-0.032414,-0.045105,0.130998,-0.279333,0.046727,0.067727,-0.024421,...,-0.013811,-0.016556,-0.004022,0.006613,0.015825,-0.006091,-0.026559,-0.013624,-0.004098,-0.018203
1993-01-04,0.036469,0.407508,0.040033,0.021855,-0.122062,0.134408,-0.092999,0.004606,0.034009,-0.025902,...,-0.030620,0.001755,-0.025642,0.001179,0.029968,0.004056,-0.013653,-0.015378,0.006626,-0.017922
1993-01-05,0.031347,0.420764,0.243270,0.098134,0.106355,0.135519,0.052505,-0.083359,-0.013644,-0.124486,...,-0.037327,0.009473,-0.023338,0.006660,0.024226,-0.010894,-0.008484,-0.016809,-0.006559,-0.000830


In [14]:
pc_1 = hv.Curve(pc_df['PC1']).opts(width=700,title ='PC1: temporal pattern of variability')
pc_2 = hv.Curve(pc_df['PC2']).opts(width=700,title ='PC2: temporal pattern of variability')
pc_3 = hv.Curve(pc_df['PC3']).opts(width=700,title ='PC3: temporal pattern of variability')
pc_4 = hv.Curve(pc_df['PC4']).opts(width=700,title ='PC4: temporal pattern of variability')
pc_5 = hv.Curve(pc_df['PC5']).opts(width=700,title ='PC5: temporal pattern of variability')

In [15]:
eig_eof_1+pc_1

:Layout
   .Overlay.I :Overlay
      .Points.I    :Points   [lon,lat]   (EOF1)
      .Coastline.I :Feature   [Longitude,Latitude]
   .Curve.I   :Curve   [index]   (PC1)

In [16]:
hv.Layout(eig_eof_2+pc_2)

:Layout
   .Overlay.I :Overlay
      .Points.I    :Points   [lon,lat]   (EOF2)
      .Coastline.I :Feature   [Longitude,Latitude]
   .Curve.I   :Curve   [index]   (PC2)

In [17]:
hv.Layout(eig_eof_3+pc_3)

:Layout
   .Overlay.I :Overlay
      .Points.I    :Points   [lon,lat]   (EOF3)
      .Coastline.I :Feature   [Longitude,Latitude]
   .Curve.I   :Curve   [index]   (PC3)

In [18]:
#eof_df, pc_df, data_from_eof_df = eof_tg(tg_data, tg_meta)

### Significance testing and variance explained

### EOF function (taking up xarray dataset)

In [19]:
if I have 

SyntaxError: invalid syntax (3703130278.py, line 1)

## EOF analysis (step by step approach)

Below is the same 

In [ ]:
dat_00 = tg_ds.sla.T.data
nt = len(tg_data.index)
covdat_00 = (dat_00.T @ dat_00)/nt 
eigvals_00, eigvecs_00 = np.linalg.eig(covdat_00)
pc_00 = dat_00 @ eigvecs_00

In [ ]:
pc_00.shape

In [ ]:
eof_names = ['EOF%s' % k for k in range(1,len(tg_ds.id)+1)]
pc_names = ['PC%s' % l for l in range(1,len(tg_ds.id)+1)]
eof_df = pd.DataFrame(eigvecs_00, index=tg_ds.id,columns = eof_names) #change eigvecs to eigvals
pc_df = pd.DataFrame(pc_00,index=tg_ds.time,columns = pc_names)

In [ ]:
pc_df

In [ ]:
val_sum0 = eigvals_00.sum()
var_exp0=np.round((eigvals_00*100/val_sum0),decimals=2) #variance explained by the 1st mode 
#we take sum of eigvals
d0 = {'EOF1':var_exp0[0],'EOF2':var_exp0[1],'EOF3':var_exp0[2],'EOF4':var_exp0[3],'rest':var_exp0[4:].sum()}
var_table0 = pd.DataFrame(d0,index=['%'])
var_table0

In [ ]:
var_exp0

### Reconstructing data from EOF

In [ ]:
data_from_eof_df = pc_df.values @ eof_df.T.values # the correct way 
data_from_eof_df = pd.DataFrame(data_from_eof_df, index = tg_ds.time, columns = tg_ds.id)

In [ ]:
data_from_eof_df

In [ ]:
data_from_eof_df['8449130'][:100].hvplot()

In [ ]:
tg_ds.sel(id='8449130',time=slice('1993-01-01','1993-11-01')).hvplot()+data_from_eof_df['8449130'][:300].hvplot()

In [ ]:
data_from_eof_df['8449130'].hvplot()*tg_ds.sel(id='8449130').hvplot().opts(line_dash='dashed',color='yellow')